In [ ]:
import os 
from google.colab import drive

drive.mount("/content/gdrive")
os.chdir("/content/gdrive/My Drive/BertTweet")


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!pip install emoji
!pip install transformers==3.1.0
import re
import emoji
import string
import json
import pandas as pd
import glob

from transformers import (
   AutoConfig,
   AutoTokenizer,
   TFAutoModelForSequenceClassification,
   AdamW,
   InputExample,
   InputFeatures
)
import tensorflow as tf
import tensorflow_datasets as tfds



     |████████████████████████████████| 890kB 7.7MB/s 
     |████████████████████████████████| 3.0MB 31.4MB/s 
     |████████████████████████████████| 1.2MB 44.3MB/s 
     |████████████████████████████████| 901kB 45.0MB/s 


# **Data Pre-processing**

In [ ]:
path = "annotated_tweets/*.csv"
csv_list = glob.glob(path)
len(csv_list)

12

In [ ]:
csv_list

['annotated_tweets/SemEval2017-task4-test.csv',
 'annotated_tweets/SemEval2017-task4-dev.csv',
 'annotated_tweets/twitter-2013train-A.csv',
 'annotated_tweets/twitter-2016train-A.csv',
 'annotated_tweets/twitter-2014test-A.csv',
 'annotated_tweets/twitter-2013test-A.csv',
 'annotated_tweets/twitter-2016devtest-A.csv',
 'annotated_tweets/twitter-2016test-A.csv',
 'annotated_tweets/twitter-2015test-A.csv',
 'annotated_tweets/twitter-2013dev-A.csv',
 'annotated_tweets/twitter-2016dev-A.csv',
 'annotated_tweets/twitter-2015train-A.csv']

In [ ]:
temp = pd.DataFrame()
for f in csv_list:
  temp = pd.concat([temp,pd.read_csv(f,sep='\t', header=None)])
temp = temp.drop(axis=1, columns=3)
temp.columns = ['id','Sentiment','Tweets']
temp = temp.reset_index()
temp.head()

,index,id,Sentiment,Tweets
0,0,801989080477154944,neutral,#ArianaGrande Ari By Ariana Grande 80% Full ht...
1,1,801989272341453952,positive,Ariana Grande KIIS FM Yours Truly CD listening...
2,2,801990978424962944,positive,Ariana Grande White House Easter Egg Roll in W...
3,3,801996232553963008,positive,#CD #Musics Ariana Grande Sweet Like Candy 3.4...
4,4,801998343442407040,neutral,SIDE TO SIDE 😘 @arianagrande #sidetoside #aria...


In [ ]:
temp.Tweets[5]

"Hairspray Live! Previews at the Macy's Thanksgiving Day Parade! https://t.co/GaFTqInolL #arianagrande #televisionnbc"

In [ ]:
df_tweet.Tweets[5]

"hairspray live previews at the macy's thanksgiving day parade arianagrande televisionnbc"

In [ ]:
temp.to_csv("data.csv")

In [ ]:
df_tweet = pd.read_csv('data.csv',index_col=0)
df_tweet.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 82621 entries, 0 to 82620
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   index      82621 non-null  int64 
 1   id         82621 non-null  int64 
 2   Sentiment  82621 non-null  object
 3   Tweets     82621 non-null  object
dtypes: int64(2), object(2)
memory usage: 3.2+ MB


In [ ]:

def clean_html(tweet):
  tweet = ' '.join(re.sub("(\w+:\/\/\S+)", " ", tweet).split())
  return tweet

def clean_hashtag(tweet):
  tweet = tweet.replace("#", " ").replace("_", " ")
  return tweet

def clean_punc(tweet):
  tweet = ' '.join(re.sub("[\.\,\!\?\:\;\-\=]", " ", tweet).split())
  return tweet

def replace_emoji(text):
    text = text.lower().strip()
    text = emoji.demojize(text, delimiters=(' ', ' '))
    return text 

def replace_at(text):
  text = text.strip()
  text = text.replace("@","at ")
  return text

def lower_case(tweet):
  tweet = tweet.lower()
  return tweet

In [ ]:
for i,row in df_tweet.iterrows():
  df_tweet['Tweets'][i] = clean_hashtag(df_tweet['Tweets'][i])
  df_tweet['Tweets'][i] = clean_html(df_tweet['Tweets'][i])
  df_tweet['Tweets'][i] = clean_punc(df_tweet['Tweets'][i])
  df_tweet['Tweets'][i] = replace_emoji(df_tweet['Tweets'][i])
  df_tweet['Tweets'][i] = lower_case(df_tweet['Tweets'][i])
  df_tweet['Tweets'][i] = replace_at(df_tweet['Tweets'][i])
  

df_tweet['Tweets']=df_tweet["Tweets"].drop_duplicates()
df_tweet = df_tweet[df_tweet['Tweets'].notna()]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from 

In [ ]:
df_tweet.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 61149 entries, 0 to 82502
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   index      61149 non-null  int64 
 1   id         61149 non-null  int64 
 2   Sentiment  61149 non-null  object
 3   Tweets     61149 non-null  object
dtypes: int64(2), object(2)
memory usage: 2.3+ MB


In [ ]:
df_tweet.Sentiment.value_counts()

neutral     27837
positive    21829
negative    11483
Name: Sentiment, dtype: int64

In [ ]:
df_tweet.to_csv("tweets_data.csv")

In [ ]:
data = pd.read_csv("tweets_data.csv")
data.head()


,Sentiment,full_text
0,positive,i'm done writing code for the week looks like ...
1,neutral,dec 21st 2012 will be know not as the end of t...
2,negative,at macmiller hate my life because i can't see ...
3,positive,checked out our lady peace at bluesfest tonigh...
4,positive,hello from the foundation trekkers we're up in...


**Dataset preparation**

In [ ]:
#Split the datset into validation, test, train sets by 1:1:5
from sklearn.model_selection import train_test_split
train_test, dev_data = train_test_split(data,test_size =1/7,random_state =42, shuffle=True)
train_data, test_data = train_test_split(train_test,test_size =1/6,random_state =42, shuffle=True)

In [ ]:
#use numbers to represent sentiments
def sentiment_label (Sentiment):
   if Sentiment == "positive":
       return 2
   elif Sentiment =="negative" :
       return 0
   else:
       return  1
train_data['Sentiment'] = train_data['Sentiment'].apply(sentiment_label)
dev_data['Sentiment'] = dev_data['Sentiment'].apply(sentiment_label)
test_data['Sentiment'] = test_data['Sentiment'].apply(sentiment_label)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [ ]:
# Choose model
# @markdown >The default model is <i><b>COVID-Twitter-BERT</b></i>. You can however choose <i><b>BERT Base</i></b> or <i><b>BERT Large</i></b> to compare these models to the <i><b>COVID-Twitter-BERT</i></b>. All these three models will be initiated with a random classification layer. If you go directly to the Predict-cell after having compiled the model, you will see that it still runs the predition. However the output will be random. The training steps below will finetune this for the specific task. <br /><br /> 
model_name = 'digitalepidemiologylab/covid-twitter-bert' #@param ["digitalepidemiologylab/covid-twitter-bert", "bert-large-uncased", "bert-base-uncased"]

# Initialise tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
def convert_data_to_examples(train, test, DATA_COLUMN, LABEL_COLUMN): 
  train_InputExamples = train.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[DATA_COLUMN], 
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)

  validation_InputExamples = test.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[DATA_COLUMN], 
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)
  
  return train_InputExamples, validation_InputExamples



def convert_examples_to_tf_dataset(examples, tokenizer, max_length=128):
    features = [] # -> will hold InputFeatures to be converted later

    for e in examples:
        # Documentation is really strong for this method, so please take a look at it
        input_dict = tokenizer.encode_plus(
            e.text_a,
            add_special_tokens=True,
            max_length=max_length, # truncates if len(s) > max_length
            return_token_type_ids=True,
            return_attention_mask=True,
            pad_to_max_length=True, # pads to the right by default # CHECK THIS for pad_to_max_length
            truncation=True
        )

        input_ids, token_type_ids, attention_mask = (input_dict["input_ids"],
            input_dict["token_type_ids"], input_dict['attention_mask'])

        features.append(
            InputFeatures(
                input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, label=e.label
            )
        )

    def gen():
        for f in features:
            yield (
                {
                    "input_ids": f.input_ids,
                    "attention_mask": f.attention_mask,
                    "token_type_ids": f.token_type_ids,
                },
                f.label,
            )

    return tf.data.Dataset.from_generator(
        gen,
        ({"input_ids": tf.int32, "attention_mask": tf.int32, "token_type_ids": tf.int32}, tf.int64),
        (
            {
                "input_ids": tf.TensorShape([None]),
                "attention_mask": tf.TensorShape([None]),
                "token_type_ids": tf.TensorShape([None]),
            },
            tf.TensorShape([]),
        ),
    )


DATA_COLUMN = 'full_text'
LABEL_COLUMN = 'Sentiment'

In [ ]:
train_InputExamples, validation_InputExamples = convert_data_to_examples(train_data, dev_data, DATA_COLUMN, LABEL_COLUMN)

train = convert_examples_to_tf_dataset(list(train_InputExamples), tokenizer)
train = train.shuffle(100).batch(6)

validation = convert_examples_to_tf_dataset(list(validation_InputExamples), tokenizer)
validation = validation.batch(6)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:1770: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


**Model Creating**

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=2e-5)

def create_model():
  config = AutoConfig.from_pretrained(model_name, num_labels=3)
  model = TFAutoModelForSequenceClassification.from_pretrained(model_name, config=config)
  model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      optimizer=opt, metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])
  return model


In [ ]:
model = create_model()

Some weights of the model checkpoint at digitalepidemiologylab/covid-twitter-bert were not used when initializing TFBertForSequenceClassification: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of TFBertForSequenceClassification were not initialized from the model checkpoint at digitalepidemiologylab/covid-twitter-bert and are newly initialized: ['dropout_147', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


**Training Model**

In [ ]:
train_data_size = len(train_data)
dev_data_size = len(dev_data)
batch_size = 6
epochs = 2

history = model.fit_generator(train, 
                    validation_data=validation, 
                    steps_per_epoch = int(train_data_size / batch_size),
                    epochs=2, 
                    verbose =1,
                    validation_steps = int(dev_data_size /batch_size))

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


7279/7279 [==============================] - 4157s 567ms/step - loss: 0.7115 - accuracy: 0.6740 - val_loss: 0.6285 - val_accuracy: 0.7169


In [ ]:
_, test_InputExamples = convert_data_to_examples(dev_data, test_data, 'full_text', 'Sentiment')

test = convert_examples_to_tf_dataset(list(test_InputExamples), tokenizer)
test = test.batch(1)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:1770: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
loss, accuracy = model.evaluate(test)

print(f'Loss: {loss}')
print(f'Accuracy: {accuracy}')

8736/8736 [==============================] - 477s 55ms/step - loss: 0.6324 - accuracy: 0.7184
Loss: 0.6324429512023926
Accuracy: 0.718406617641449


# **Evaluation**

In [ ]:
#after loading tokenizer and model

def format_prediction(ds):
  pred_sentences = []
  pred_sentences = list(ds['full_text'])
  #for index,row in ds:
    #pred_sentences.append(row['Tweets'])
  tf_batch = tokenizer(pred_sentences, max_length=128, padding=True, truncation=True, return_tensors='tf')
  tf_outputs = model(tf_batch)
  tf_predictions = tf.nn.softmax(tf_outputs[0], axis=-1)
  label = tf.argmax(tf_predictions, axis=1)
  label = label.numpy()
  new = pd.concat([ds,pd.DataFrame(label,columns=['pred'])],axis=1)
  return new

In [ ]:
ts = pd.read_csv('TEST.csv')

In [ ]:
ts.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 900 entries, 22132 to 44693
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Sentiment  900 non-null    int64 
 1   full_text  900 non-null    object
dtypes: int64(1), object(1)
memory usage: 21.1+ KB


In [ ]:
pred = format_prediction(ts)

In [ ]:
labels = ['Negative','Neutral','Positive']
from sklearn.metrics import classification_report

print(classification_report(pred['Sentiment'],pred['pred'], target_names = labels))

# **Save Model**

In [ ]:
model.save_pretrained("ct_bert")


# **Predict**

In [ ]:
model = TFAutoModelForSequenceClassification.from_pretrained("ct_bert")
# Choose model
# @markdown >The default model is <i><b>COVID-Twitter-BERT</b></i>. You can however choose <i><b>BERT Base</i></b> or <i><b>BERT Large</i></b> to compare these models to the <i><b>COVID-Twitter-BERT</i></b>. All these three models will be initiated with a random classification layer. If you go directly to the Predict-cell after having compiled the model, you will see that it still runs the predition. However the output will be random. The training steps below will finetune this for the specific task. <br /><br /> 
model_name = 'digitalepidemiologylab/covid-twitter-bert' #@param ["digitalepidemiologylab/covid-twitter-bert", "bert-large-uncased", "bert-base-uncased"]

# Initialise tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

Some weights of the model checkpoint at ct_bert were not used when initializing TFBertForSequenceClassification: ['dropout_73']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of TFBertForSequenceClassification were not initialized from the model checkpoint at ct_bert and are newly initialized: ['dropout_221']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
#after loading tokenizer and model
labels = ['Negative','Positive']
def format_prediction(ds):
  pred_sentences = []
  pred_sentences = list(ds['Tweets'])
  #for index,row in ds:
    #pred_sentences.append(row['Tweets'])
  tf_batch = tokenizer(pred_sentences, max_length=280, padding=True, truncation=True, return_tensors='tf')
  tf_outputs = model(tf_batch)
  tf_predictions = tf.nn.softmax(tf_outputs[0], axis=-1)
  label = tf.argmax(tf_predictions, axis=1)
  label = label.numpy()
  new = pd.concat([ds,pd.DataFrame(label,columns=['pred'])],axis=1)
  return new
  

In [ ]:
df = pd.read_csv('Geo_tagged/2020_5.csv')
df_tweet = pd.DataFrame()
df_tweet['time']  = df['created_at']
df_tweet["Tweets"] = df["full_text"]
df_tweet['id'] = df['id_str']
df_tweet['place'] = df['place']
df_tweet=df_tweet.drop_duplicates()
df_tweet

,time,Tweets,id,place
0,Fri May 01 16:50:50 +0000 2020,"Before, during and after #COVID19, achieving #...",1256264828743757826,"{'id': 'b71fac2ee9792cbe', 'url': 'https://api..."
1,Sat May 02 13:17:34 +0000 2020,The Pioneer DDJ-SZ is one of my favorite decks...,1256573546886815744,"{'id': '36763957ba75d166', 'url': 'https://api..."
2,Sat May 02 16:23:44 +0000 2020,We designed a vulnerability survey to provide ...,1256620399753859076,"{'id': '000c69ad123213a8', 'url': 'https://api..."
3,Sat May 02 17:43:03 +0000 2020,"Distant But Connected.\n\nJust like this, we a...",1256640360731860992,"{'id': '272983f6b52c196e', 'url': 'https://api..."
4,Sat May 02 18:39:40 +0000 2020,More lines today. Looks like social distancin...,1256654607281008641,"{'id': 'bc38440673d47ddf', 'url': 'https://api..."
...,...,...,...,...
41561,Mon Jun 01 01:46:06 +0000 2020,I am the same all year round always about our ...,1267271170312867840,"{'id': '4de072969805ac41', 'url': 'https://api..."
41562,Wed May 27 12:11:52 +0000 2020,15-17 May 2020 we #livestreamed the first ever...,1265616710046035969,"{'id': '3eb2c704fe8a50cb', 'url': 'https://api..."
41563,Thu May 07 03:15:56 +0000 2020,How we will all feel when isolation is over. #...,1258234081504112640,"{'id': '01864a8a64df9dc4', 'url': 'https://api..."
41564,Fri May 22 17:05:53 +0000 2020,❤❤ I challenge you to #SaveTheWorld from #Covi...,1263878763747717120,"{'id': '01a9a39529b27f36', 'url': 'https://api..."
